In [1]:
import numpy as np, pandas as pd, os,sys
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import colorConverter
import gmplot
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, Birch, MiniBatchKMeans,DBSCAN
from sklearn import metrics

from scipy.spatial.distance import cdist,pdist
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

# %matplotlib inline

In [2]:
OUTPUT_FOLDER = "output"
## july-oct-raw2-train.csv
# csv_file = "july-oct-batch2-train.csv" ## raw_input("Enter csv file to load:")
# csv_file = "batch2-train.csv" ## raw_input("Enter csv file to load:")
csv_file = "july-oct-batch3-train.csv" ## raw_input("Enter csv file to load:")
file_str = "%s/%s" %(OUTPUT_FOLDER, csv_file)
a = pd.read_csv(file_str) 
a.columns = ['index1', 'address','city', 'day','hour','type','latitude','longitude','parent_incident','state']
a.drop('index1',axis=1,inplace=True)
a.describe()
X = a[['address','city', 'day','hour','type','latitude','longitude','parent_incident','state']] ## MODEL 1
# X = a[['address', 'day','hour','type','parent_incident','latitude','longitude']] ## MODEL 2
# X = a[['address', 'day','hour','type','parent_incident','latitude','longitude']] ## MODEL 2

pca = PCA(n_components=2).fit(X)
pca_2d = pd.DataFrame(pca.transform(X))

# Add PCA cols to DF
X['pca1'] = pca_2d[0]
X['pca2'] = pca_2d[1]

In [ ]:

# fig, (ax1) = plt.subplots(1)
# fig.set_size_inches(16, 6)
# ax1.scatter(X['pca1'],X['pca2'],c=X['type'],s=5,alpha=0.8)

#### K-MEANS LOOP

In [181]:
ari_list = np.array([])

x_axis = np.arange(2,4,1)
for i in x_axis:
    ## RUN K-MEANS
    km = KMeans(n_clusters=i, init='k-means++').fit(pca_2d)
    km_pred = KMeans(n_clusters=10, init='k-means++').fit_predict(pca_2d)
    labels = km.labels_
    clust_centers = km.cluster_centers_
    ## GET ARI 
    ari = metrics.adjusted_rand_score(km.labels_, km_pred) 
    ari_list = np.append(ari_list, ari)
    sil = metrics.silhouette_score(pca_2d, km_pred)
    ## PLOT
    title_str = "K-Means, Clusters=%s, ARI=%.3f, Silhouette Coefficient=%.3f" %(i,ari,sil)
    fig, (ax1) = plt.subplots(1)
    fig.set_size_inches(12, 4)
    points = ax1.scatter(pca_2d[0],pca_2d[1],c=labels,s=5, alpha=0.6)
    centroids = ax1.scatter(clust_centers[:,1],clust_centers[:,0], marker='o',s=75, edgecolors='w')
    ax1.legend([points,centroids],['Points','Centroids'])
    ax1.set_title(title_str)
    ax1.set_ylabel("PCA 1")
    ax1.set_xlabel("PCA 2")

# for i in x_axis:
fig, (ax2) = plt.subplots(1)
fig.set_size_inches(16, 6)  
ari_score = ax2.scatter(x_axis,ari_list,c='r',s=40)
ax2.plot(x_axis,ari_list,c='b')
ax2.legend([ari_score],['Adjusted Rand Score'])
ax2.set_title("Adjusted Rand Index for K-Means")
ax2.set_ylabel("Adjusted Rand Score")
ax2.set_xlabel("Clusters")
plt.show()

In [177]:
CLUSTER_NUMBER = 3
km = KMeans(n_clusters=CLUSTER_NUMBER, init='k-means++').fit(pca_2d)
km_pred = KMeans(n_clusters=10, init='k-means++').fit_predict(pca_2d)
labels = km.labels_
clust_centers = km.cluster_centers_
# print len(labels)
clust_centers





array([[-1.28425115,  0.35885024],
       [ 1.28696969,  1.39095928],
       [ 0.31068367, -0.84364178]])

In [192]:
from sklearn.metrics import pairwise_distances_argmin_min
raw_map_df = pd.read_csv("output/july-oct-raw.csv")
a_centroids = pd.DataFrame([])
gmap_centroids = pd.DataFrame([])
for i in clust_centers:
    closest, y = pairwise_distances_argmin_min(i, pca_2d)
    a_centroids = a_centroids.append(a.ix[closest[0]])
    gmap_centroids = a_centroids.append(raw_map_df.ix[closest[0]])
    
plt.scatter(a['longitude'],a['latitude'],c=a['klabel'],s=3,alpha=0.5)
plt.scatter(a_centroids['longitude'],a_centroids['latitude'],c='r',s=150, marker='o',edgecolors='w', alpha=0.8)

## GENERATE GOOGLE MAP IN SEPARATE HTML FILE



gmap = gmplot.GoogleMapPlotter(29.8, -95.4, 9.0)
gmap.scatter(raw_map_df['latitude'], raw_map_df['longitude'], '#3B0B39', alpha=0.4, size=60, marker=False)
gmap.heatmap(gmap_centroids['latitude'], gmap_centroids['longitude'],radius=(20))

## PATH IS OUTPUT FOLDER/FILENAME + SEQUENCE NUMBER.html
map_file = "%s/%s-%s.html" %(OUTPUT_FOLDER,"KMEANS",CLUSTER_NUMBER)
gmap.draw(map_file)
plt.show()


/Users/edwingarcia/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/edwingarcia/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/edwingarcia/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contain

#### BIRCH


In [ ]:
## BIRCH CLUSTERING TOOL
x_axis = np.arange(0.3,0.9,0.1)

for i in x_axis:
    ## LOOP OVER THRESHOLD
    brc = Birch(branching_factor = 50, n_clusters = 3, threshold = i, compute_labels = True)
    brc.fit(pca_2d)
    

    labels = brc.labels_
    pred_labels = brc.predict(pca_2d)
    centroids = brc.subcluster_centers_
    centroids_size = len(centroids)
    


    fig, (ax1)= plt.subplots(1, figsize=(16,8))
    points = ax1.scatter(pca_2d[0],pca_2d[1],c=labels,s=5, alpha=0.6)
    centroids = ax1.scatter(centroids[:,0],centroids[:,1],marker='^',s=75)
    title_str = "BIRCH, Threshold=%s, Centroids = %s" %(i,centroids_size)
    ax1.legend([points,centroids],['Points','Centroids'])
    ax1.set_title(title_str)
    ax1.set_ylabel("PCA 1")
    ax1.set_xlabel("PCA 2")
    

plt.show()
# print len(labels)

### MINIBATCH

In [ ]:
# cluster_nums = int(raw_input("Number of clusters: "))
batch_size = int(raw_input("Batch size (default 100): "))
ari_list = np.array([])
x_axis = np.arange(2,5,1)

for i in x_axis:
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=i, batch_size=batch_size)

    mbk.fit(pca_2d)
    mbk_pred = mbk.fit_predict(pca_2d)
    clust_centers = mbk.cluster_centers_
    labels = mbk.labels_
    
    ## GET ARI
    ari = metrics.adjusted_rand_score(labels, mbk_pred) 
    ari_list = np.append(ari_list, ari)
    print ari, labels,mbk_pred
    

    fig, (ax1)= plt.subplots(1, figsize=(16,8))
    points = ax1.scatter(pca_2d[0],pca_2d[1],c=labels,s=5, alpha=0.6)
    centroids = ax1.scatter(clust_centers[:,0],clust_centers[:,1],marker='o', edgecolors='w', s=75)
    title_str = "MiniBatch K-Means, Batch Size=%s, Clusters = %s" %(batch_size,i)
    ax1.set_title(title_str)
    ax1.legend([points,centroids],['Points','Centroids'])
    ax1.set_ylabel("PCA 1")
    ax1.set_xlabel("PCA 2")
    
plt.show()

### DBSCAN

In [3]:
epsilon = 0.08
min_samples=10

eps_list = np.arange(.09,.1,0.02)
min_samples_list = np.arange(18,20,3)
for i in eps_list:
    
    for j in min_samples_list:
        db = DBSCAN(eps=i, min_samples=j).fit(pca_2d)
        labels = db.labels_
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        col_set = pd.Series(list(set(labels)))
        clusters = pd.DataFrame([])
        for k in db.core_sample_indices_:
        #     print X.loc[i]
            if k > -1:
                clusters = clusters.append(X.loc[k])
                
        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))       
        
        fig, (ax1,ax2)= plt.subplots(1,2, figsize=(16,6))
        percent_clustered = len(clusters)/float(len(labels)) * 100
        
        points = ax1.scatter(pca_2d[0],pca_2d[1],c=colors,s=5, marker='.', alpha=0.4)
        points = ax2.scatter(pca_2d[0],pca_2d[1],c=labels,s=20, marker='.', alpha=0.4)
        centroids = ax2.scatter(clusters['pca1'],clusters['pca2'], c=colors, marker='o',s=10, alpha=0.7)
        
        title_str = "DBSCAN, Epsilon=%s, min_samples=%s\nClusters=%s, Clustered points=%.2f" %(i,j,n_clusters_,percent_clustered)
        ax1.legend([points,centroids],['Points','Clusters'])
        ax1.set_title(title_str)
        ax1.set_ylabel("PCA 1")
        ax1.set_xlabel("PCA 2")
        
        title_str2 = "DBSCAN, Epsilon=%s, min_samples=%s\nClusters=%s, Clustered points=%.2f%%" %(i,j,n_clusters_,percent_clustered)
        ax2.legend([points,centroids],['Points','Clusters'])
        ax2.set_title(title_str2)
        ax2.set_ylabel("PCA 1")
        ax2.set_xlabel("PCA 2")
        print "eps:",i,", min_samples:",j,", Clusters: ",n_clusters_,", Clustered Points:", len(clusters)

plt.show()

eps: 0.09 , min_samples: 18 , Clusters:  19 , Clustered Points: 8705


In [ ]:
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = 'k'

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

### DIAGNOSTICS

#### Elbow Test

In [3]:
# %matplotlib 


def plot_variance(start_,end_,interval_=1):
    plt.subplots(figsize=(12,5))
    
#     X = X.as_matrix()
    k_list = np.arange(start_,end_+1,interval_)

    k_var = [KMeans(n_clusters=k).fit(pca_2d) for k in k_list]

    centroids = [c.cluster_centers_ for c in k_var]

    k_euclidian = [cdist(pca_2d, cent, 'euclidean') for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclidian]

    ## Total Within cluster SS
    wcss = [sum(d**2) for d in dist]
    
    ## Total SS
    tss = sum(pdist(pca_2d)**2) / pca_2d.shape[0]

    ## Between cluster SS
    bss = tss - wcss    
    print bss 
    plt.title("KMeans Explained Variance")
    plt.plot(k_list,bss)
    points = plt.scatter(k_list,bss,c='r', s=25)
    plt.ylabel('Variance')
    plt.xlabel('K-value')
   
    plt.grid('on', which='major', axis='x' )
    plt.grid('on', which='major', axis='y' )
   
    plt.legend([points],['Variance'])  
    plt.xticks(k_list)

start_k = int(raw_input("Enter starting K: "))
end_k = int(raw_input("Enter ending K: "))

plot_variance(start_k, end_k)
plt.show()

Enter starting K: 2
Enter ending K: 10
[ 11927.41456607  21367.59518396  24295.74548767  26342.28404651
  27954.37065979  28819.34007813  29577.05531618  30133.20817567
  30680.53037466]
